In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/santander-customer-satisfaction/sample_submission.csv
/kaggle/input/santander-customer-satisfaction/train.csv
/kaggle/input/santander-customer-satisfaction/test.csv


### 1. Preparações
Dado os resultados anteriores, a abordagem do Oversampling e Undersampling também será testada com todas as colunas viáveis do conjunto de dados, para avaliar o impacto da dimensionalidade.

In [47]:
dataset = pd.read_csv("/kaggle/input/santander-customer-satisfaction/train.csv")

Eliminando as colunas constantes.

In [48]:
colunas_constantes = [coluna for coluna in dataset.columns if dataset[coluna].nunique() == 1]
dataset = dataset.drop(columns = colunas_constantes)

In [4]:
dataset.shape

(76020, 337)

In [5]:
ID = dataset.ID
dataset = dataset.drop(columns = "ID")

In [6]:
TARGET = dataset.TARGET
dataset = dataset.drop(columns = "TARGET")

In [7]:
dataset.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000
1,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000
2,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000
3,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000
4,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016


#### 1.1 Escalando o Conjunto

In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

dataset_scaled = scaler.fit_transform(dataset.copy())

dataset_scaled = pd.DataFrame(dataset_scaled, columns = dataset.columns)
dataset_scaled.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,0.999764,0.18,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001545
1,0.999764,0.29,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002003
2,0.999764,0.18,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002822
3,0.999764,0.32,0.0,0.01513,0.009275,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002671
4,0.999764,0.34,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005091


#### 1.2 Undersampling e Oversampling

In [9]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

SEED = 912039

In [11]:
smote = SMOTE(random_state = SEED)
x_smote, y_smote = smote.fit_resample(dataset_scaled, TARGET)
dataset_smote = pd.concat([x_smote, y_smote], axis = 1)
dataset_smote.TARGET.value_counts()

TARGET
0    73012
1    73012
Name: count, dtype: int64

In [13]:
sampler = RandomUnderSampler(random_state = SEED)
x_under, y_under = sampler.fit_resample(dataset_scaled, TARGET)
dataset_under = pd.concat([x_under, y_under], axis = 1)
dataset_under.TARGET.value_counts()

TARGET
0    3008
1    3008
Name: count, dtype: int64

In [18]:
y_oversampling = dataset_smote.TARGET
y_undersampling = dataset_under.TARGET

dataset_smote.drop("TARGET", axis = 1, inplace = True)
dataset_under.drop("TARGET", axis = 1, inplace = True)

### 2. Treinamento
Devido a alta dimensionalidade, é possível que alguns modelos demorem bastante para treinamento. Em contrapartida, entretanto, não haverão vários conjuntos de dados a serem treinados, já que não existe uma necessidade para escolher colunas.

In [14]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

linear = SVC(random_state = SEED)
knn = KNeighborsClassifier()
lr = LogisticRegression(random_state = SEED)
dt = DecisionTreeClassifier(random_state = SEED)
rf = RandomForestClassifier(random_state = SEED)
ab = AdaBoostClassifier(random_state = SEED)
gb = GradientBoostingClassifier(random_state = SEED)
extra = ExtraTreesClassifier(random_state = SEED)
mlp = MLPClassifier(random_state = SEED)

In [15]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, roc_auc_score

def avalia_modelo(modelo, x, y):
    scoring = {
        'accuracy': 'accuracy',
        'precision': 'precision',
        'recall': 'recall',
        'f1': 'f1',
        'roc_auc': make_scorer(roc_auc_score)
    }
    
    resultados = cross_validate(modelo, x, y, scoring=scoring, cv=10, n_jobs=-1)
    
    print(f'Modelo: {modelo}')
    media_acuracia = sum(resultados['test_accuracy'])/10
    media_precisao = sum(resultados['test_precision'])/10
    media_recall = sum(resultados['test_recall'])/10
    media_f1 = sum(resultados['test_f1'])/10
    media_roc_auc = sum(resultados['test_roc_auc'])/10
    
    print(f'Acurácia --> {media_acuracia :.2f}')
    print(f'Precisão --> {media_precisao :.2f}')
    print(f'Recall --> {media_recall :.2f}')
    print(f'F1 --> {media_f1 :.2f}')
    print(f'ROC AUC --> {media_roc_auc :.2f}')

#### 4.1 KNN

In [19]:
avalia_modelo(knn, dataset_under, y_undersampling)

Modelo: KNeighborsClassifier()
Acurácia --> 0.69
Precisão --> 0.69
Recall --> 0.68
F1 --> 0.68
ROC AUC --> 0.69


O resultado, para o conjunto de Undersampling, no KNN, já é superior ao uso de Undersampling na Segunda Tentativa.

In [20]:
avalia_modelo(knn, dataset_smote, y_oversampling)

Modelo: KNeighborsClassifier()
Acurácia --> 0.86
Precisão --> 0.85
Recall --> 0.88
F1 --> 0.87
ROC AUC --> 0.86


Apesar dos mais de 4 minutos de execução, o KNN também apresentou uma melhoria de resultado se em comparação até mesmo com a primeira tentativa.

#### 4.2 Árvore de Decisão

In [21]:
avalia_modelo(dt, dataset_under, y_undersampling)

Modelo: DecisionTreeClassifier(random_state=912039)
Acurácia --> 0.67
Precisão --> 0.67
Recall --> 0.67
F1 --> 0.67
ROC AUC --> 0.67


In [22]:
avalia_modelo(dt, dataset_smote, y_oversampling)

Modelo: DecisionTreeClassifier(random_state=912039)
Acurácia --> 0.89
Precisão --> 0.88
Recall --> 0.90
F1 --> 0.89
ROC AUC --> 0.89


Na árvore de decisão, o conjunto de dados maior parece ter um efeito contrário. Ou ele compromete a performance ou ele é irrelevante.

#### 4.3 Regressão Logística

In [23]:
avalia_modelo(lr, dataset_under, y_undersampling)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Modelo: LogisticRegression(random_state=912039)
Acurácia --> 0.70
Precisão --> 0.69
Recall --> 0.74
F1 --> 0.71
ROC AUC --> 0.70


In [24]:
avalia_modelo(lr, dataset_smote, y_oversampling)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Modelo: LogisticRegression(random_state=912039)
Acurácia --> 0.72
Precisão --> 0.70
Recall --> 0.76
F1 --> 0.73
ROC AUC --> 0.72


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


A Regressão Logística também não variou muito em performance.

#### 4.4 RandomForest

In [25]:
avalia_modelo(rf, dataset_under, y_undersampling)

Modelo: RandomForestClassifier(random_state=912039)
Acurácia --> 0.73
Precisão --> 0.75
Recall --> 0.71
F1 --> 0.73
ROC AUC --> 0.73


In [26]:
avalia_modelo(rf, dataset_smote, y_oversampling)

/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Modelo: RandomForestClassifier(random_state=912039)
Acurácia --> 0.90
Precisão --> 0.92
Recall --> 0.89
F1 --> 0.90
ROC AUC --> 0.90


#### 4.5 GradientBoosting

In [28]:
avalia_modelo(gb, dataset_under, y_undersampling)

Modelo: GradientBoostingClassifier(random_state=912039)
Acurácia --> 0.76
Precisão --> 0.77
Recall --> 0.76
F1 --> 0.76
ROC AUC --> 0.76


In [29]:
avalia_modelo(gb, dataset_smote, y_oversampling)

Modelo: GradientBoostingClassifier(random_state=912039)
Acurácia --> 0.87
Precisão --> 0.87
Recall --> 0.86
F1 --> 0.87
ROC AUC --> 0.87


#### 4.6 VotingClassifier
Por último, a fim de tentar corrigir erros de outros modelos, vamos tentar um VotingClassifier

In [30]:
from sklearn.ensemble import VotingClassifier

vc = VotingClassifier(
    estimators=[
        ('LogisticRegression', lr),
        ('RandomForest', rf),
        ('KNN', knn),
        ('GradientBoosting', gb),
        ('MLP', mlp),
    ],
    voting='hard'  # 'hard' para classificação (votação por maioria) ou 'soft' para votação ponderada pelas probabilidades
)

In [32]:
avalia_modelo(vc, dataset_under, y_undersampling)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Modelo: VotingClassifier(estimators=[('LogisticRegression',
                              LogisticRegression(random_state=912039)),
                             ('RandomForest',
                              RandomForestClassifier(random_state=912039)),
                             ('KNN', KNeighborsClassifier()),
                             ('GradientBoosting',
                              GradientBoostingClassifier(random_state=912039)),
                             ('MLP', MLPClassifier(random_state=912039))])
Acurácia --> 0.76
Precisão --> 0.76
Recall --> 0.74
F1 --> 0.75
ROC AUC --> 0.76


In [33]:
avalia_modelo(vc, dataset_smote, y_oversampling)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Modelo: VotingClassifier(estimators=[('LogisticRegression',
                              LogisticRegression(random_state=912039)),
                             ('RandomForest',
                              RandomForestClassifier(random_state=912039)),
                             ('KNN', KNeighborsClassifier()),
                             ('GradientBoosting',
                              GradientBoostingClassifier(random_state=912039)),
                             ('MLP', MLPClassifier(random_state=912039))])
Acurácia --> 0.88
Precisão --> 0.87
Recall --> 0.90
F1 --> 0.89
ROC AUC --> 0.88


### 3. Teste

In [46]:
alvo = pd.read_csv("/kaggle/input/santander-customer-satisfaction/test.csv")

alvo_id = alvo.ID
alvo.drop(columns = "ID", inplace = True)

scaler = MinMaxScaler()

copia = alvo.copy()
copia = scaler.fit_transform(copia)

alvo = pd.DataFrame(copia, columns = alvo.columns)
alvo.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,0.999764,0.27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001361
1,0.999764,0.30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001533
2,0.999764,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001585
3,0.999764,0.19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.006462
4,0.999764,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.002507


In [35]:
modelo_over = VotingClassifier(
    estimators=[
        ('LogisticRegression', lr),
        ('RandomForest', rf),
        ('KNN', knn),
        ('GradientBoosting', gb),
        ('MLP', mlp),
    ],
    voting='hard'  # 'hard' para classificação (votação por maioria) ou 'soft' para votação ponderada pelas probabilidades
)

modelo_under = VotingClassifier(
    estimators=[
        ('LogisticRegression', lr),
        ('RandomForest', rf),
        ('KNN', knn),
        ('GradientBoosting', gb),
        ('MLP', mlp),
    ],
    voting='hard'  # 'hard' para classificação (votação por maioria) ou 'soft' para votação ponderada pelas probabilidades
)

In [36]:
modelo_over.fit(dataset_smote, y_oversampling)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


VotingClassifier(estimators=[('LogisticRegression',
                              LogisticRegression(random_state=912039)),
                             ('RandomForest',
                              RandomForestClassifier(random_state=912039)),
                             ('KNN', KNeighborsClassifier()),
                             ('GradientBoosting',
                              GradientBoostingClassifier(random_state=912039)),
                             ('MLP', MLPClassifier(random_state=912039))])

In [37]:
modelo_under.fit(dataset_under, y_undersampling)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


VotingClassifier(estimators=[('LogisticRegression',
                              LogisticRegression(random_state=912039)),
                             ('RandomForest',
                              RandomForestClassifier(random_state=912039)),
                             ('KNN', KNeighborsClassifier()),
                             ('GradientBoosting',
                              GradientBoostingClassifier(random_state=912039)),
                             ('MLP', MLPClassifier(random_state=912039))])

In [49]:
alvo = alvo.drop(columns = colunas_constantes)

In [50]:
alvo.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,0.999764,0.27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001361
1,0.999764,0.30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001533
2,0.999764,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.001585
3,0.999764,0.19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.006462
4,0.999764,0.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000004,0.000004,0.0,0.0,0.0,0.0,0.002507


In [51]:
resultados_over = modelo_over.predict(alvo)
resposta_over = pd.concat([alvo_id, pd.Series(resultados_over, name = "TARGET")], axis = 1)
resposta_over

,ID,TARGET
0,2,1
1,5,0
2,6,0
3,7,0
4,9,0
...,...,...
75813,151831,1
75814,151832,0
75815,151833,0
75816,151834,0


In [52]:
resultados_under = modelo_under.predict(alvo)
resposta_under = pd.concat([alvo_id, pd.Series(resultados_under, name = "TARGET")], axis = 1)
resposta_under

,ID,TARGET
0,2,1
1,5,1
2,6,0
3,7,0
4,9,0
...,...,...
75813,151831,0
75814,151832,0
75815,151833,0
75816,151834,1


In [53]:
resposta_over.TARGET.value_counts()

TARGET
0    62456
1    13362
Name: count, dtype: int64

In [54]:
resposta_under.TARGET.value_counts()

TARGET
0    53788
1    22030
Name: count, dtype: int64

In [55]:
resposta_over.to_csv("resposta_over.csv", index = False)
resposta_under.to_csv("resposta_under.csv", index = False)